In [1]:
detection_model_path = '/media/prateek/prateek_space/intel_model_dump/intel/vehicle-detection-adas-0002/FP32/vehicle-detection-adas-0002.xml'
attr_model_path = '/media/prateek/prateek_space/intel_model_dump/intel/vehicle-attributes-recognition-barrier-0039/FP32/vehicle-attributes-recognition-barrier-0039.xml'
cpu_extension = '../build_samples/intel64/Release/lib/libcpu_extension.so'
device = 'CPU'


In [2]:
import time,cv2
import numpy as np
import IPython.display as Disp
import support_utility_openvino

In [3]:
vehicle_detection = support_utility_openvino.async_infer(4)
vehicle_detection.load_model(cpu_exension_path=cpu_extension,model_path=detection_model_path,device=device)

attr_detection = support_utility_openvino.async_infer()
attr_detection.load_model(cpu_exension_path=cpu_extension,model_path=attr_model_path,device=device)

output_support = support_utility_openvino.create_plot()

loading sample element
loading sample element


In [4]:
vehicle_colors =  ["white", "gray", "yellow", "red", "green", "blue", "black"]
vehicle_type = ["car", "bus", "truck", "van"]

In [6]:
channel = "/media/prateek/prateek_space/dataset/china_number_plate_detection.mp4"
cap = cv2.VideoCapture(channel)
cv2.namedWindow("preview",cv2.WINDOW_NORMAL)
ret,frame = cap.read()
fps = []
thresh = .5
# dets = []
while True:
    fps = fps[-100:]
#     Disp.clear_output(wait=True)
    ret,frame = cap.read()
    if not ret:
        break
    t1 = time.time()
    vehicle_detection.predict(frame)
    if vehicle_detection.frame_processed>vehicle_detection.buffer_shape:
        try:
            frame,attr,res = vehicle_detection.postprocess_op()
            op,bboxes = output_support.trim_frame_with_result(frame,res[0],threshold=thresh)
            attrs = [attr_detection.predict_sync(vehicle) for vehicle in op]
            attrs = [(vehicle_colors[np.argmax(att[0])], vehicle_type[np.argmax(att[1])]) for att in attrs]
#             dets.append(len(attrs))
            fps.append(1/(time.time()-t1))
            op_frame = output_support.write_text(frame,"FPS = "+str(np.mean(fps,dtype='int')),text_color = (0,0,0),font_scale=1,font_thickness=2,highlight_color = (127,0,255))
            for idx,b in enumerate(bboxes):
                op_frame = output_support.create_bbox_with_text(op_frame,(tuple(b[:2]),tuple(b[2:])),",".join(attrs[idx]))
#             op_frame = output_support.label_obj_detection(frame,res[0],labels=['vehicle','person','car'],font_scale=1,font_thickness = 1,thresh=.5)
        except Exception as e:
            print(e)
#         print(vehicle_detection.cursor_id,vehicle_detection.frame_processed)
        cv2.imshow("preview",op_frame)
        k = cv2.waitKey(1)
        if k==27:
            break
cap.release()
cv2.destroyAllWindows()
        
        
    